In [87]:
import numpy as np
import pandas as pd

#learning
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

#preprocessing images
import cv2


## Preprocessing and loading in the data

In [88]:
# the values per image that belong to the given attributes for each class
attribute_values = np.load('feather-in-focus/attributes.npy')

# names of each bird class
labels = np.load("feather-in-focus/class_names.npy", allow_pickle=True).item()

# list of training data containing: image location, labels per image 
data = pd.read_csv("feather-in-focus/train_images.csv")


In [89]:
# code that adds the attributes per label to the training_df
att_list = []
for i in range(len(data)):
    att_list.append(attribute_values[data['label'][i]-1])  
    
data['attributes'] = att_list

In [98]:
def load_and_preprocess_images(image_paths, file_path):
    images = []
    for img_path in image_paths:
        img = cv2.imread(f'feather-in-focus/{file_path}{img_path}' )
        img = cv2.resize(img, (224, 224))  # Resize the image
        img = img / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
    return np.array(images)


In [99]:
# Load and preprocess images
X_images = load_and_preprocess_images(data['image_path'], 'train_images')
y_labels = to_categorical(data['label'].values)

X_train, X_test, y_train, y_test = train_test_split(X_images, y_labels, test_size=0.2, random_state=0)

## Building the model


In [ ]:
# Create Xception model with pre-trained weights
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3), classes=200)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for classification
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10  # Adjust as needed
model.fit(np.array(X_train), y_train, epochs=epochs, validation_data=(np.array(X_test), y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(np.array(X_test), y_test)
print(f'Test accuracy: {test_acc}')

## predicting the submission images and saving it for submission 


In [101]:

test_df = pd.read_csv("feather-in-focus/test_images_path.csv")
test_images = load_and_preprocess_images(test_df['image_path'],'test_images')

# Predict
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)


submission_df = pd.DataFrame({
    'id': test_df['id'],
    'label': predicted_labels
})
submission_df.to_csv('submission.csv', index=False)

125/125 [==============================] - 73s 579ms/step
